In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from pandas.plotting import register_matplotlib_converters
import japanize_matplotlib 
from arch import arch_model

# yfinanceを使用して日経225のデータを取得
NK225_data = yf.download('^N225', start='任意の日付', end=datetime.now().strftime('%Y-%m-%d'))

# 'Adj Close'列のデータを取得
NK225_prices = NK225_data['Adj Close']

# 欠損値を削除
NK225_prices = NK225_prices.dropna()

# データをCSVファイルに保存
NK225_prices.to_csv('NK225_prices.csv')

# CSVファイルからデータを読み込む
NK225_prices = pd.read_csv('NK225_prices.csv', index_col='Date', parse_dates=True)


# GARCHモデルの作成とトレーニング
model = arch_model(NK225_prices, vol='Garch', p=1, q=1)  # 適切な次数を選択する必要
result = model.fit()

# 未来のボラティリティを予測
forecast_steps = 30
forecast = result.forecast(horizon=forecast_steps)

# 未来の日付に対するボラティリティの予測
forecast_values = forecast.variance.values[-1, :]

# 日付のリストを作成
future_dates = pd.date_range(start=NK225_prices.index[-1], periods=forecast_steps + 1, freq='B')[1:]

# 未来のボラティリティを可視化
plt.figure(figsize=(10, 6))
plt.plot(NK225_prices.index, NK225_prices, label='実際の価格', color='blue')
plt.plot(future_dates, forecast_values, label='未来のボラティリティ予測', color='red', linestyle='--')
plt.fill_between(future_dates, 0, forecast_values, color='red', alpha=0.1)  # ボラティリティの範囲を塗りつぶす
plt.title('未来のボラティリティ予測')
plt.xlabel('日付')
plt.ylabel('価格/ボラティリティ')
plt.legend()
plt.show()